In [ ]:
from polytope.api import Client
import earthkit.data
from earthkit.regrid import interpolate

EMAIL = "destine_370a@deltares.nl"
KEY = "f78021d45305407b702b541e3f22da99"

In [ ]:
request = {
    'activity': 'ScenarioMIP',
    'class': 'd1',
    'dataset': 'climate-dt',
    'date': '20241102/to/20241104',
    'experiment': 'SSP3-7.0',
    'expver': '0001',
    'generation': '1',
    'levtype': 'sfc',
    'model': 'IFS-NEMO',
    'param': '134/165/166',
    'realization': '1',
    'resolution': 'standard',
    'stream': 'clte',
    'time': '0000/0100/0200/0300/0400/0500/0600/0700/0800/0900/1000/1100/1200/1300/1400/1500/1600/1700/1800/1900/2000/2100/2200/2300', # '0100/0200/0300/0400/0500/0600'
    'type': 'fc',
}

In [ ]:
c = Client(address='polytope.lumi.apps.dte.destination-earth.eu', user_email=EMAIL, user_key=KEY)
c.revoke('all')
c.retrieve('destination-earth', request, f'test_dt_climate_fetch_large.grib')

In [ ]:
data = earthkit.data.from_source('file', "test_dt_climate_fetch.grib")

In [ ]:
data.ls()

In [ ]:
data_latlon = earthkit.regrid.interpolate(data, out_grid={"grid": [.1,.1]}, method='linear')
ds = data_latlon.to_xarray()

## Read, interpolate and to netcdf

In [ ]:
import hydromt

In [ ]:
bounds = [-15.0, 43.0, 13.0, 63.0]

In [ ]:
data = earthkit.data.from_source('file', "dt-climate-winter-3839.grib").sel(dataDate=slice(20390207,20390220))

In [ ]:
data.ls()

In [ ]:
data = interpolate(data, out_grid={"grid": [.1,.1]}, method="linear")
ds = data.to_xarray(xarray_open_dataset_kwargs={"chunks": {"time": 1}}).squeeze()

In [ ]:
ds = ds.assign_coords(
    {"longitude": ((ds.longitude+180)%360)-180}
)

ds = ds.sortby("longitude")
ds = ds.sortby("latitude")

ds = ds.rename(
    {
        "longitude": "x",
        "latitude": "y",
        "sp": "press_msl",
        "u10": "wind10_u",
        "v10": "wind10_v"
    }
)

In [ ]:
ds.raster.set_crs(4326)

In [ ]:
ds = ds.sel(
    x = slice(bounds[0], bounds[2]),
    y = slice(bounds[1], bounds[3])
)

In [ ]:
ds.to_netcdf("dt-climate_event_feb2039.nc")

In [ ]:
import xarray as xr

In [ ]:
ds_new = xr.open_dataset("dt-climate_event_feb2039.nc")